In [ ]:
import numpy as np
import scipy as sp
from scipy import stats
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

plt.rcParams.update({'font.size': 16})

In [ ]:
res_behav_data = pd.read_csv('../data/RESTRICTED_esfinn_11_21_2021_19_19_35.csv')
res_behav_data.set_index("Subject", inplace=True)
res_behav_data.index = res_behav_data.index.map(str)
print(res_behav_data.shape)
res_behav_data.head()

Read in unrestricted behavioral data:

In [ ]:
unres_behav_data = pd.read_csv('../data/unrestricted_esfinn_11_21_2021_19_19_13.csv')
unres_behav_data.set_index("Subject", inplace=True)
unres_behav_data.index = unres_behav_data.index.map(str)
print(unres_behav_data.shape)
unres_behav_data.head()

In [ ]:
behav_data = pd.concat([res_behav_data, unres_behav_data], axis=1)
print(behav_data.shape)
behav_data.head()

Read in FMRI data, join into single dataframe, then join this with the larger dataframe:

In [ ]:
data_file_loc = '/Users/f0053cz/Dropbox (Dartmouth College)/postdoc_Dartmouth/HCP/fMRIScipts/data'
sub_id_all = np.load(os.path.join(data_file_loc,'sub_ID_all.npy'))
sub_id_all = [str(i) for i in sub_id_all]
print(len(sub_id_all),sub_id_all[:5])

# load behavioral responses too - to identify missed/invalid trials later - 1 for "social", 0 for "nonsocial" and 9 for "unsure", nan for missed response
responses = np.load(os.path.join(data_file_loc,'responses.npy'))
responses.shape # subs *movies

In [ ]:
for r in [0,1,9]:
    print(len(np.where(responses[:,4]==r)[0]))
print(len(np.where(np.isnan(responses[:,4]))[0]))
print(587+88+186+187)

- lots of missing responses in RANDOM MECH - let's keep this here for now

In [ ]:
shen268_lbl = pd.read_csv(os.path.join(data_file_loc,"shen_dictionary.csv"))
pd.set_option('display.max_colwidth', None)
shen268_lbl.head()

In [ ]:
# for comparison with the other main figure if needed
nodes_coaxbill_rand_all = np.load(os.path.join(data_file_loc,'nodes_coaxbill_rand_all.npy')) # sig nodes hihglighted in the first S>NS GLM
len(np.where(nodes_coaxbill_rand_all)[0])

In [ ]:
from multipy.fdr import lsu
from corrstats import independent_corr

In [ ]:
vidnames = ["COAXING-B", "BILLIARD-A", "DRIFTING-A", "Fishing", "Random mechanical","Scaring", "SEDUCING-B", "STAR-A", "SURPRISING-B", "TENNIS-A"]


# traits vs. fmri unmodulated by responses

In [ ]:
def get_correlations(coef_slope, movie_ind,sub_id_all,responses,corrtype):
    
    corr_nodes,   pval_nodes    =  [np.empty((268,)) for _ in range(2)] # nodes * responses (non-social, social)
    corr_nodes[:],pval_nodes[:] = [np.nan] *2

    for n in range(268): 
        if n% 50 ==0:
            print('node',n)

        fMRI_data = pd.DataFrame({'Subject':sub_id_all,'slopeReg_node':coef_slope[:,n], 'responses':responses[:,movie_ind]})
        fMRI_data.set_index("Subject", inplace=True)
        data = fMRI_data.join(behav_data, how='inner') # join betas and trait info

        x,y = "slopeReg_node", "ASR_Intn_T" #y = "Loneliness_Unadj"
        
        inds = ~np.isnan(data[x]) & ~np.isnan(data[y]) #& ~np.isnan(data['responses'])# find rows where neither x or y is NaN
        
        # Calculate and print correlations
        if corrtype == 'Pearson':
            corr_nodes[n], pval_nodes[n] = stats.pearsonr(data[x][inds], data[y][inds])
        elif corrtype =='Spearman':
            corr_nodes[n], pval_nodes[n] = stats.spearmanr(data[x][inds], data[y][inds])
            

    pval_nodes_fdr = lsu(pval_nodes,q=.05)

    return corr_nodes, pval_nodes, pval_nodes_fdr

In [ ]:
# movie "SURPRISING-B, most social

movie_ind = 8
coef_slope = np.load(os.path.join(data_file_loc,'coefs_trial_norm','slope_reg',f'coef_slopereg_trialnorm_{vidnames[movie_ind]}.npy'))

corr_nodes_surp, pval_nodes_surp, pval_nodes_fdr_surp = get_correlations(coef_slope, movie_ind, sub_id_all, responses, 'Spearman')


In [ ]:
# movie "COAXING-B, second most social

movie_ind = 0
coef_slope = np.load(os.path.join(data_file_loc,'coefs_trial_norm','slope_reg',f'coef_slopereg_trialnorm_{vidnames[movie_ind]}.npy'))

corr_nodes_coax, pval_nodes_coax, pval_nodes_fdr_coax = get_correlations(coef_slope, movie_ind, sub_id_all, responses, 'Spearman')


In [ ]:
# movie DRIFTING, most nonsocial

movie_ind = 2
coef_slope = np.load(os.path.join(data_file_loc,'coefs_trial_norm','slope_reg',f'coef_slopereg_trialnorm_{vidnames[movie_ind]}.npy'))

corr_nodes_drif, pval_nodes_drif, pval_nodes_fdr_drif = get_correlations(coef_slope, movie_ind, sub_id_all, responses, 'Spearman')


In [ ]:
# movie DRIFTING, most nonsocial

movie_ind = 1
coef_slope = np.load(os.path.join(data_file_loc,'coefs_trial_norm','slope_reg',f'coef_slopereg_trialnorm_{vidnames[movie_ind]}.npy'))

corr_nodes_bill, pval_nodes_bill, pval_nodes_fd_bill = get_correlations(coef_slope, movie_ind, sub_id_all, responses, 'Spearman')


In [ ]:
'''print('Surp')
print('fdr',np.where(pval_nodes_fdr_surp)[0].shape)
print('unc.',np.where(pval_nodes_surp<.05)[0].shape)

print('\ncoax')
print('fdr',np.where(pval_nodes_fdr_coax))
print('unc.',np.where(pval_nodes_coax<.05),pval_nodes_surp[pval_nodes_coax<.05])


print('\nsurp-coax intersection')
inter_ = np.where((pval_nodes_surp<.05)&(pval_nodes_coax<.05))
corr_nodes_coax[inter_],corr_nodes_surp[inter_]
# 3 nodes intersect, but different signs..'''

- for SURPRISING-B, many nodes survive mult-comp correction, but not other movies (taken individually).
- Trait vs. fMRI, nothing survives at the whole brain level
- but nevertheless, looks like the correlation is negative - but this could be driven by lesser activation or more deactivation  which means different things - so I looked at the mean/median beta in the sig nodes


In [ ]:
#plt.hist(np.nanmean(coef_slope,axis=0),label='all nodes')
plt.hist(np.nanmean(coef_slope[:,pval_nodes_surp<.05],axis=0),label='Mean slope beta, nodes\nwith sig. correl (p<.05)', color = [.7,.7,.7])
plt.hist(np.nanmean(coef_slope[:,pval_nodes_fdr_surp],axis=0),label='Mean slope beta, nodes\nwith sig correl (q<.05)',color = [.3,.3,.3])
ymin,ymax =  plt.ylim()
plt.vlines(np.nanmean(coef_slope[:,pval_nodes_surp<.05]),ymin,ymax,color= 'k',ls='dashed',label='Mean, p<.05 nodes')
plt.vlines(np.nanmean(coef_slope[:,pval_nodes_fdr_surp]),ymin,ymax, color = 'k',ls='solid',label='Mean, q<.05 nodes')
plt.legend(bbox_to_anchor =[1.01,1])
plt.xlabel('Mean slope beta per node')
plt.ylabel('Number of nodes')
plt.title('SURPRISING-B')


t,p=stats.ttest_1samp(coef_slope[:,np.where(pval_nodes_fdr_surp)[0]],0,axis=0,nan_policy='omit')
print(max(p))

t,p=stats.ttest_1samp(coef_slope[:,np.where(pval_nodes_surp<.05)[0]],0,axis=0,nan_policy='omit')
print(max(p))

t,p=stats.ttest_1samp(coef_slope,0,axis=0,nan_policy='omit')
print(np.where(p>.05))

plt.ylim(0,30)
plt.xlim(-2,2)

# for all of the orange and green nodes, mean sig >0, and most nodes across all 268 too - so the negative correlations do show lower activation 

In [ ]:
plt.figure(1,figsize=(10,10))
plt.suptitle('SURPRISING-B, trait-fMRI (slope beta) correlation coefft\nacross all 268 nodes')
plt.subplot(121)
plt.hist(corr_nodes_surp,bins=20,orientation='horizontal',color='grey')
_, pval = stats.ttest_1samp(corr_nodes_surp,0)
avg  = np.nanmean(corr_nodes_surp)
xmin,xmax = plt.xlim()
plt.hlines(avg,xmin,xmax,color = 'orange',label = f'Mean= {avg:.2f},\n(p={pval:.1e})')
plt.ylim(-1,1)
plt.ylabel('Spearman correlation coefft',fontsize=20)
plt.xlabel('Nr. of nodes')
plt.title('Histogram')
plt.legend()


plt.subplot(122)
plt.boxplot(corr_nodes_surp,vert=True)
plt.xticks([])
plt.ylim(-1,1)
plt.title('Boxplot')
plt.ylabel('Spearman correlation coefft',fontsize=20)
plt.xlim(.8,1.2)
plt.tight_layout()

# Plot brain maps for the movie above

In [ ]:
from nltools.data import Brain_Data
from nltools.mask import expand_mask, roi_to_brain
from nilearn.plotting import plot_glass_brain, plot_surf_roi,plot_stat_map,plot_img,plot_surf_contours
from nilearn import datasets
from nilearn.surface import vol_to_surf

import nibabel as nib

from nilearn.datasets import fetch_surf_fsaverage
fsaverage = fetch_surf_fsaverage()

#coords = [10*int(i) for i in np.linspace(-4,7,6)]
bg_img = datasets.load_mni152_template()

mask = Brain_Data('https://neurovault.org/media/images/8423/shen_2mm_268_parcellation.nii.gz')
mask_x = expand_mask(mask)


def color_rois(values):
    
    """
    This function assumes you are passing a vector "values" with the same length as the number of nodes in the atlas.
    """
    shen268 = nib.load(os.path.join(data_file_loc,"shen_2mm_268_parcellation.nii.gz"))
    shen268_data = shen268.get_fdata()
    img = np.zeros(shen268_data.shape)
    #print(shen268_data.shape)
    for roi in range(len(values)):
        itemindex = np.where(shen268_data==roi+1) # find voxels in this node (add 1 to account for zero-indexing)
        #print(len(itemindex[0]))
        img[itemindex] = values[roi] # color them by the desired value 

    affine = shen268.affine
    img_nii = nib.Nifti1Image(img, affine)
    
    return img_nii

fig_save_loc = os.path.join('/Users/f0053cz/Dropbox (Dartmouth College)/postdoc_Dartmouth/HCP/paper_prep/figures/fig2_2_traits/fmri_results')

In [ ]:
print('all nodes:',min(corr_nodes_surp),max(corr_nodes_surp))
print('sig unc nodes:',min(corr_nodes_surp[pval_nodes_surp<.05]),max(corr_nodes_surp[pval_nodes_surp<.05]))

In [ ]:
def plot_result_brainmaps(nodes, vmin, vmax, txt, title_txt, contours_mat, pval_, intersection_, params,text_x):
    cmap = 'RdBu_r'

    levels,labels,colors,coods = params['levels'],params['labels'],params['colors'],params['coords']

    fig,ax = plt.subplots(nrows=1,ncols=4,figsize = (20,5),subplot_kw={'projection': '3d'})
    
    img = roi_to_brain(pd.Series(nodes), mask_x)
    ax_plot = plot_img(img.to_nifti(), display_mode = 'z',vmin = vmin, vmax = vmax, cut_coords = coords,cmap = cmap, bg_img = bg_img,colorbar= False,threshold = .0001)
    #ax_plot.add_contours(color_rois((nodes_coaxbill_rand_all)),linewidths=1, colors=colors_glm_nodes,linestyles='solid')
    ax_plot.add_contours(color_rois((pval_)),linewidths=1, colors=[colors[0]],linestyles='solid')
    #ax_plot.add_contours(color_rois((nodes_coaxbill_rand_all)),linewidths=1, colors=[colors[1]],linestyles='solid')
    ax_plot.add_contours(color_rois((intersection_)),linewidths=1, colors=[colors[1]],linestyles='solid')
    #ax_plot.title(title_txt)
    #ax_plot._colorbar_ax.text(1,.8*vmax,txt,fontsize=20,fontdict = {'verticalalignment':'top','rotation':0})#get_legend()
    #plt.savefig(os.path.join(fig_save_loc,f'RAND/{resp_lbl[resp]}/axial.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
    #plt.clf()


    #LH
    texture = vol_to_surf(color_rois(nodes), fsaverage.pial_left,interpolation='nearest',radius =1, n_samples=1)
    surf_plot1=plot_surf_roi(fsaverage.infl_left, texture, hemi='left',cmap = cmap, colorbar=False, symmetric_cmap=True, vmin = vmin, vmax = vmax,
                                bg_map=fsaverage.sulc_left,axes=ax[0], title=title_txt)#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
    texture_contour = vol_to_surf(color_rois(contours_mat), fsaverage.pial_left,interpolation='nearest',radius =1, n_samples=1)
    plot_surf_contours(fsaverage.infl_left, texture_contour, axes = ax[0], figure=surf_plot1, legend=True,levels = levels, colors=colors)

    
    surf_plot2=plot_surf_roi(fsaverage.infl_left, texture, hemi='left',cmap = cmap, colorbar=False, symmetric_cmap=True, vmin = vmin, vmax = vmax,
                                bg_map=fsaverage.sulc_left, view = 'medial',axes=ax[1])#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
    plot_surf_contours(fsaverage.infl_left, texture_contour, axes = ax[1], figure=surf_plot2, legend=True,levels = levels, colors=colors)


    #RH
    texture = vol_to_surf(color_rois(nodes), fsaverage.pial_right,interpolation='nearest',radius =1, n_samples=1)
    surf_plot3=plot_surf_roi(fsaverage.infl_right, texture, hemi='right',cmap = cmap, colorbar=True,symmetric_cmap=True, vmin = vmin, vmax = vmax,
                                bg_map=fsaverage.sulc_right,axes=ax[3])#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
    surf_plot3.axes[4].text(text_x,.5*vmax,s=txt,fontsize=20, fontdict = {'verticalalignment':'top','horizontalalignment':'left','rotation':0})
    texture_contour = vol_to_surf(color_rois(contours_mat), fsaverage.pial_right,interpolation='nearest',radius =1, n_samples=1)
    plot_surf_contours(fsaverage.infl_right, texture_contour, axes = ax[3], figure=surf_plot3, legend=True,levels = levels, labels = labels, colors=colors)

    surf_plot4=plot_surf_roi(fsaverage.infl_right, texture, hemi='right',cmap = cmap, colorbar=False,symmetric_cmap=True, vmin = vmin, vmax = vmax,
                                bg_map=fsaverage.sulc_right, view ='medial',axes=ax[2])#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
    plot_surf_contours(fsaverage.infl_right, texture_contour, axes = ax[2], figure=surf_plot4, legend=True,levels = levels, colors=colors)
    #plt.savefig(os.path.join(fig_save_loc,f'RAND/{resp_lbl[resp]}/right_med.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
    #plt.clf()


In [ ]:
min(corr_nodes_surp),max(corr_nodes_surp)

In [ ]:
# surprising
coords = [-50,-40,-25,-10,0,10,25,40,55] # initial exploration
#coords = [-40,-30,-20] # for the final version
cmap = 'RdBu_r'
   
vmin,vmax = -.16,.16

#txt = r"$\overline{\beta}(''Social''-''Non-social'')$"
txt = 'Spearman r (ASR_Intn_T vs.\nslope beta)'

nodes = np.zeros((268,))
nodes[pval_nodes_surp<.05] = corr_nodes_surp[pval_nodes_surp<.05]
movie_ind = 8
title_txt = f'{vidnames[movie_ind]} (nodes unc.)'

contours_mat = np.zeros((268,))
contours_mat[pval_nodes_fdr_surp] = 1
#contours_mat[nodes_coaxbill_rand_all] = 2
contours_mat[nodes_int_sign_fdr] = 2

levels = [1,2]
labels = ['q<.05','intersection']
colors = [[0,0,0,1],[1,0,0,1]]
params = {'levels':levels, 'labels': labels, 'colors': colors, 'coords':coords}
text_x=2
plot_result_brainmaps(nodes,vmin,vmax,txt,title_txt,contours_mat,pval_nodes_fdr_surp,nodes_int_sign_fdr,params,text_x)


In [ ]:
def explore_correlation(x, y, data,corrtype='Spearman'):
    """
    Calculates and plots correlation between x and y variables in dataframe `data`, plus distribution of x and y 
    """
    sns.set_style("white")
    
    inds = ~np.isnan(data[x]) & ~np.isnan(data[y]) # find rows where neither x or y is NaN

    g = sns.jointplot(x=x, y=y, data=data, kind='reg', color='gray')

    # Calculate and print correlations
    if corrtype == 'Pearson':
        rp, pp = stats.pearsonr(data[x][inds], data[y][inds])
        g.ax_joint.annotate(f'r_p = {rp:.2f} (p={pp:.1g})', xy=(.05,.95), xycoords='axes fraction')
    elif corrtype == 'Spearman':
        rs, ps = stats.spearmanr(data[x][inds], data[y][inds])
        g.ax_joint.annotate(f'r_s = {rs:.2f} (p={ps:.1g})', xy=(.05,.95), xycoords='axes fraction')
    
    return g

In [ ]:
vidnames

In [ ]:
#plt.figure(1,figsize = (15,5))
#plt.subplot(121)
corrtype = 'Pearson'

df = pd.DataFrame({'corr_nodes_surp':corr_nodes_surp,'corr_nodes_coax':corr_nodes_coax})
explore_correlation('corr_nodes_surp','corr_nodes_coax',df,corrtype)
plt.xlim(-.15,.15)
plt.ylim(-.15,.15)
ax = plt.gca()
ax.set_aspect('equal')
plt.xticks([-.15,-.1,-.05,0,.05,.1,.15])
plt.yticks([-.15,-.1,-.05,0,.05,.1,.15])
plt.xlabel('r SURPRISING-B ("Social")')
plt.ylabel('r COAXING-B ("Social")')
plt.scatter(corr_nodes_surp[nodes_int_sign_fdr],corr_nodes_coax[nodes_int_sign_fdr],color='k', label = 'SURP &&\nALL OTHERS, q<.05')
plt.legend(bbox_to_anchor = [2.01,1])

#plt.subplot(122)
df = pd.DataFrame({'corr_nodes_surp':corr_nodes_surp,'corr_nodes_drif':corr_nodes_drif})
explore_correlation('corr_nodes_surp','corr_nodes_drif',df,corrtype)
plt.xlim(-.15,.15)
plt.ylim(-.15,.15)
ax = plt.gca()
ax.set_aspect('equal')
plt.xticks([-.15,-.1,-.05,0,.05,.1,.15])
plt.yticks([-.15,-.1,-.05,0,.05,.1,.15])
plt.xlabel('r SURPRISING-B ("Social")')
plt.ylabel('r DRIFTING-A ("Non-social")')
plt.scatter(corr_nodes_surp[nodes_int_sign_fdr],corr_nodes_drif[nodes_int_sign_fdr],color='k', label = 'SURP & ALL OTHERS, q<.05')


#plt.subplot(122)
df = pd.DataFrame({'corr_nodes_surp':corr_nodes_surp,'corr_nodes_bill':corr_nodes_bill})
explore_correlation('corr_nodes_surp','corr_nodes_bill',df,corrtype)
plt.xlim(-.15,.15)
plt.ylim(-.15,.15)
ax = plt.gca()
ax.set_aspect('equal')
plt.xticks([-.15,-.1,-.05,0,.05,.1,.15])
plt.yticks([-.15,-.1,-.05,0,.05,.1,.15])
plt.xlabel('r SURPRISING-B ("Social")')
plt.ylabel('r BILLIARDS-A ("Non-social")')
plt.scatter(corr_nodes_surp[nodes_int_sign_fdr],corr_nodes_bill[nodes_int_sign_fdr],color='k', label = 'SURP & ALL OTHERS, q<.05')


df = pd.DataFrame({'corr_nodes_bill':corr_nodes_bill,'corr_nodes_drif':corr_nodes_drif})
explore_correlation('corr_nodes_bill','corr_nodes_drif',df),corrtype
plt.xlim(-.15,.15)
plt.ylim(-.15,.15)
ax = plt.gca()
ax.set_aspect('equal')
plt.xticks([-.15,-.1,-.05,0,.05,.1,.15])
plt.yticks([-.15,-.1,-.05,0,.05,.1,.15])
plt.xlabel('r BILLIARDS-A ("Non-social")')
plt.ylabel('r DRIFTING-A ("Non-social")')


In [ ]:
np.where(pval_nodes_surp<.05)

In [ ]:
# print ROIs neatly
import pprint
pd.set_option('display.max_colwidth', 300)
pd.set_option('display.precision', 3)

nodes = np.where(pval_nodes_fdr_surp)[0]#_surp<.05)[0]
print(len(nodes))
# writing our nodes for RANDOM MECH
pd.DataFrame({'Node': [n+1 for n in nodes], 'rcoef(Sp)':corr_nodes_surp[nodes],'pval(Sp)':pval_nodes_surp[nodes], 'ROI name':[eval(shen268_lbl[str(n+1)][0])['name'] for n in nodes]}).sort_values(by=['rcoef(Sp)'],
ascending = False)

In [ ]:
# nodes which show a diff between corr"Social" and corr"Non-social" for RANODM MECH and across nodes in the main GLM analysis
print('overlap with the main GLM nodes:', np.where(nodes_coaxbill_rand_all & (pval_nodes_surp<.05))) 

# ALL MOVIES

In [ ]:
# first of all, best to remove subjects with < 10 responses for power in general
count_resp = np.zeros((responses.shape[0],))
for i in range(responses.shape[0]):
    count_resp[i] = len(np.where(~np.isnan(responses[i,:]))[0])
subs_10resp = np.where(count_resp == 10)[0]
len(subs_10resp)

In [ ]:
# creating a 3D array of beta coeffs across movies from individual movie files
all_coefs_run = np.zeros((1048,268,10))
all_coefs_trial = np.zeros((1048,268,10))

vidnames = ["COAXING-B", "BILLIARD-A", "DRIFTING-A", "Fishing", "Random mechanical","Scaring", "SEDUCING-B", "STAR-A", "SURPRISING-B", "TENNIS-A"]
for m in range(10):
    fileName =  os.path.join(data_file_loc,'coefs_trial_norm','slope_reg',f'coef_slopereg_trialnorm_{vidnames[m]}.npy')
    all_coefs_run[:,:,m] = np.load(fileName)

    #fileName =  os.path.join(data_file_loc,'coefs_run_norm','slope_reg',f'coef_slopereg_runnorm_{vidnames[m]}.npy')
    #all_coefs_trial[:,:,m] = np.load(fileName)
    #print(dat.shape)

In [ ]:
from pymer4.models import Lmer,Lm
import time

In [ ]:
# ALL MOVIES - LME - run norm slope betas

coef_nodes_all_lme_run,   pval_nodes_all_lme_run   =  [np.empty((268,)) for _ in range(2)] # nodes * responses (non-social, social)
coef_nodes_all_lme_run[:],pval_nodes_all_lme_run[:] = [np.nan] *2

#coef_nodes_all_lr_run,   pval_nodes_all_lr_run   =  [np.empty((268,)) for _ in range(2)] # nodes * responses (non-social, social)
#coef_nodes_all_lr_run[:],pval_nodes_all_lr_run[:] = [np.nan] *2


nresp = [len(np.where(~np.isnan(responses[i,:]))[0]) for i in range(responses.shape[0])]
start_time = time.time()
for n in range(268): 
    if n% 50 ==0:
        print('node',n, ', time taken=',time.time()-start_time)

    data = pd.DataFrame({})#columns = ['Subject','coef','response','movie'])
    for m in range(10):
        #fMRI_data = pd.DataFrame({'coefs': all_coefs[subs_10resp,n,m],'response':responses[subs_10resp,m], 'subID':sub_id_all[subs_10resp], 'movie':np.repeat(m,len(subs_10resp))})
        fMRI_data = pd.DataFrame({'Subject':sub_id_all,'slopeReg_node': all_coefs_run[:,n,m], 'response': responses[:,m],'nresp':nresp, 'movie': np.repeat(m,1048)})
        #fMRI_data = fMRI_data.loc[fMRI_data['nresp']==10,:]
        fMRI_data.set_index("Subject", inplace=True)
        data_temp = fMRI_data.join(behav_data['ASR_Intn_T'], how='inner') # join betas and trait info
        data = data.append(data_temp)#,ignore_index=True)
    data.reset_index(inplace=True)

    inds = ~np.isnan(data['slopeReg_node']) & ~np.isnan(data['ASR_Intn_T']) # find rows where neither x or y is NaN
    data = data.loc[inds,:]
    # Calculate and print correlations
    
    #model1 = Lm('slopeReg_node ~  ASR_Intn_T ', data=data) # ff: mean response, rf:subjID
    #model1.fit(summary = False, verbose = False, no_warnings = True)
    model2 = Lmer('slopeReg_node ~  ASR_Intn_T + (1|movie)', data=data) # ff: mean response, rf:subjID
    model2.fit(summary = False, verbose = False, no_warnings = True)
    #model.fit(factors={"response":["Non-social","Social"]},summary = False, verbose = False)

    #if (len(model1.warnings) == 0):
    #    coef_nodes_all_lr_run[n] = model1.coefs['Estimate'][1]
    #    pval_nodes_all_lr_run[n] = model1.coefs['P-val'][1]

    if (len(model2.warnings) == 0):
        coef_nodes_all_lme_run[n] = model2.coefs['Estimate'][1]
        pval_nodes_all_lme_run[n] = model2.coefs['P-val'][1]
   
#pval_nodes_fdr_all_lr_run = lsu(pval_nodes_all_lr_run,q=.05)
pval_nodes_fdr_all_lme_run = lsu(pval_nodes_all_lme_run,q=.05)
np.where(pval_nodes_fdr_all_lme_run)#, np.where(pval_nodes_fdr_all_lr_run)


In [ ]:
len(np.where(nodes_coaxbill_rand_all&pval_nodes_fdr_all_lme_run)[0])

In [ ]:
# ALL OTHER MOVIES - LME - run norm slope betas

coef_nodes_allother_lme_run,   pval_nodes_allother_lme_run   =  [np.empty((268,)) for _ in range(2)] # nodes * responses (non-social, social)
coef_nodes_allother_lme_run[:],pval_nodes_allother_lme_run[:] = [np.nan] *2

#coef_nodes_all_lr_run,   pval_nodes_all_lr_run   =  [np.empty((268,)) for _ in range(2)] # nodes * responses (non-social, social)
#coef_nodes_all_lr_run[:],pval_nodes_all_lr_run[:] = [np.nan] *2


nresp = [len(np.where(~np.isnan(responses[i,:]))[0]) for i in range(responses.shape[0])]
start_time = time.time()
for n in range(268): 
    if n% 50 ==0:
        print('node',n, ', time taken=',time.time()-start_time)

    data = pd.DataFrame({})#columns = ['Subject','coef','response','movie'])
    for m in [0,1,2,3,4,5,6,7,9]:#range(10):
        #fMRI_data = pd.DataFrame({'coefs': all_coefs[subs_10resp,n,m],'response':responses[subs_10resp,m], 'subID':sub_id_all[subs_10resp], 'movie':np.repeat(m,len(subs_10resp))})
        fMRI_data = pd.DataFrame({'Subject':sub_id_all,'slopeReg_node': all_coefs_run[:,n,m], 'response': responses[:,m],'nresp':nresp, 'movie': np.repeat(m,1048)})
        #fMRI_data = fMRI_data.loc[fMRI_data['nresp']==10,:]
        fMRI_data.set_index("Subject", inplace=True)
        data_temp = fMRI_data.join(behav_data['ASR_Intn_T'], how='inner') # join betas and trait info
        data = data.append(data_temp)#,ignore_index=True)
    data.reset_index(inplace=True)

    inds = ~np.isnan(data['slopeReg_node']) & ~np.isnan(data['ASR_Intn_T']) # find rows where neither x or y is NaN
    data = data.loc[inds,:]
    # Calculate and print correlations
    
    #model1 = Lm('slopeReg_node ~  ASR_Intn_T ', data=data) # ff: mean response, rf:subjID
    #model1.fit(summary = False, verbose = False, no_warnings = True)
    model2 = Lmer('slopeReg_node ~  ASR_Intn_T + (1|movie)', data=data) # ff: mean response, rf:subjID
    model2.fit(summary = False, verbose = False, no_warnings = True)
    #model.fit(factors={"response":["Non-social","Social"]},summary = False, verbose = False)

    #if (len(model1.warnings) == 0):
    #    coef_nodes_all_lr_run[n] = model1.coefs['Estimate'][1]
    #    pval_nodes_all_lr_run[n] = model1.coefs['P-val'][1]

    if (len(model2.warnings) == 0):
        coef_nodes_allother_lme_run[n] = model2.coefs['Estimate'][1]
        pval_nodes_allother_lme_run[n] = model2.coefs['P-val'][1]
   
#pval_nodes_fdr_all_lr_run = lsu(pval_nodes_all_lr_run,q=.05)
pval_nodes_fdr_allother_lme_run = lsu(pval_nodes_allother_lme_run,q=.05)
np.where(pval_nodes_fdr_allother_lme_run)#, np.where(pval_nodes_fdr_all_lr_run)


In [ ]:
#plt.hist(np.nanmean(all_coefs_run,axis=(0,2)),label='all nodes')
plt.hist(np.nanmean(all_coefs_run[:,pval_nodes_all_lme_run<.05,:],axis=(0,2)),label='Mean slope beta, nodes\nwith sig. correl (p<.05)',color = [.7,.7,.7])
plt.hist(np.nanmean(all_coefs_run[:,pval_nodes_fdr_all_lme_run,:],axis=(0,2)),label='Mean slope beta, nodes\nwith sig correl (q<.05)',color = [.3,.3,.3])
ymin,ymax =  plt.ylim()
plt.vlines(np.nanmean(all_coefs_run[:,pval_nodes_all_lme_run<.05,:]),ymin,ymax,color= 'k',ls='dashed',label='Mean, p<.05 nodes')
plt.vlines(np.nanmean(all_coefs_run[:,pval_nodes_fdr_all_lme_run,:]),ymin,ymax, color = 'k',ls='solid',label='Mean, q<.05 nodes')

plt.legend(bbox_to_anchor =[1.01,1])
plt.xlabel('Mean slope beta per node')
plt.ylabel('Number of nodes')
plt.title('ALL MOVIES')

plt.xlim(-2,2)
plt.ylim(0,30)
coefs_ = np.array([np.ndarray.flatten(all_coefs_run[:,n,:]) for n in np.where(pval_nodes_fdr_all_lme_run)[0]]).T
t,p=stats.ttest_1samp(coefs_,0,axis=0,nan_policy='omit')
print(max(p))

coefs_ = np.array([np.ndarray.flatten(all_coefs_run[:,n,:]) for n in np.where(pval_nodes_all_lme_run<.05)[0]]).T
t,p=stats.ttest_1samp(coefs_,0,axis=0,nan_policy='omit')
print(max(p))

coefs_ = np.array([np.ndarray.flatten(all_coefs_run[:,n,:]) for n in range(268)]).T
t,p=stats.ttest_1samp(coefs_,0,axis=0,nan_policy='omit')
print(np.where(p>.05)[0])

# for all of the orange and green nodes, mean sig >0, and most nodes across all 268 too - so the negative correlations do show lower activation 

In [ ]:
print(len(np.where(pval_nodes_fdr_all_lme_run)[0]), len(np.where(pval_nodes_all_lme_run<.05)[0]))
print(len(np.where(pval_nodes_fdr_allother_lme_run)[0]),len(np.where(pval_nodes_allother_lme_run<.05)[0]))
print(len(np.where(pval_nodes_fdr_surp)[0]), len(np.where(pval_nodes_surp<.05)[0]))

nodes_int_sign_fdr = pval_nodes_fdr_surp & pval_nodes_fdr_allother_lme_run & (np.sign(corr_nodes_surp) == np.sign(coef_nodes_allother_lme_run))
nodes_int_sign_unc = (pval_nodes_surp<.05) & (pval_nodes_allother_lme_run<.05) & (np.sign(corr_nodes_surp) == np.sign(coef_nodes_allother_lme_run))
print(len(np.where(nodes_int_sign_fdr)[0]), len(np.where(nodes_int_sign_unc)[0]))


In [ ]:
max(corr_nodes_surp[ (pval_nodes_surp<.05) & (pval_nodes_allother_lme_run<.05) ])

In [ ]:
max(coef_nodes_allother_lme_run[ (pval_nodes_surp<.05) & (pval_nodes_allother_lme_run<.05) ])

In [ ]:
nodes = np.where(pval_nodes_fdr_surp & pval_nodes_fdr_allother_lme_run)[0]
print(len(nodes))
# writing our nodes for RANDOM MECH
pd.DataFrame({'Node': [n+1 for n in nodes], 'rcoef(Sp)':corr_nodes_surp[nodes],'pval(Sp)':pval_nodes_surp[nodes], 'ROI name':[eval(shen268_lbl[str(n+1)][0])['name'] for n in nodes]}).sort_values(by=['rcoef(Sp)'],
ascending = False)

In [ ]:
#ALL OTHER MOVIES
#plt.hist(np.nanmean(all_coefs_run,axis=(0,2)),label='all nodes')
other_movies = [0,1,2,3,4,5,6,7,9]
arr_ = all_coefs_run[:,:,other_movies]
plt.hist(np.nanmean(arr_[:,pval_nodes_allother_lme_run<.05,:],axis=(0,2)),label='Mean slope beta, nodes\nwith sig. correl (p<.05)',color = [.7,.7,.7])
plt.hist(np.nanmean(arr_[:,pval_nodes_fdr_allother_lme_run,:],axis=(0,2)),label='Mean slope beta, nodes\nwith sig correl (q<.05)',color = [.3,.3,.3])

ymin,ymax =  plt.ylim()
plt.vlines(np.nanmean(arr_[:,pval_nodes_all_lme_run<.05,:]),ymin,ymax,color= 'k',ls='dashed',label='Mean, p<.05 nodes')
plt.vlines(np.nanmean(arr_[:,pval_nodes_fdr_all_lme_run,:]),ymin,ymax, color = 'k',ls='solid',label='Mean, q<.05 nodes')

plt.legend(bbox_to_anchor =[1.01,1])
plt.xlabel('Mean slope beta per node')
plt.ylabel('Numnber of nodes')
plt.title('ALL OTHER MOVIES')

plt.xlim(-2,2)
plt.ylim(0,30)

coefs_ = np.array([np.ndarray.flatten(all_coefs_run[:,n,other_movies]) for n in np.where(pval_nodes_fdr_allother_lme_run)[0]]).T
t,p=stats.ttest_1samp(coefs_,0,axis=0,nan_policy='omit')
print(max(p))

coefs_ = np.array([np.ndarray.flatten(all_coefs_run[:,n,other_movies]) for n in np.where(pval_nodes_allother_lme_run<.05)[0]]).T
t,p=stats.ttest_1samp(coefs_,0,axis=0,nan_policy='omit')
print(max(p))

coefs_ = np.array([np.ndarray.flatten(all_coefs_run[:,n,other_movies]) for n in range(268)]).T
t,p=stats.ttest_1samp(coefs_,0,axis=0,nan_policy='omit')
print(np.where(p>.05)[0])

# for all of the orange and green nodes, mean sig >0, and most nodes across all 268 too - so the negative correlations do show lower activation 

In [ ]:
print(len(np.where(pval_nodes_fdr_all_lme_run)[0]), 'nodes')

plt.figure(1,figsize=(10,5))
plt.suptitle('ALL MOVIES, trait-fMRI (slope beta) reg coefft\nacross all 268 nodes')
plt.subplot(121)
plt.hist(coef_nodes_all_lme_run,bins=20,orientation='horizontal',color='grey')
_, pval = stats.ttest_1samp(coef_nodes_all_lme_run,0)
avg  = np.nanmean(coef_nodes_all_lme_run)
xmin,xmax = plt.xlim()
plt.hlines(avg,xmin,xmax,color = 'orange',label = f'Mean= {avg:.4f},\n(p={pval:.1e})')
#plt.ylim(-1,1)
plt.ylabel('lme  coefft',fontsize=20)
plt.xlabel('Nr. of nodes')
plt.title('Histogram')
plt.ylim(-.006,.006)
plt.legend()


plt.subplot(122)
plt.boxplot(coef_nodes_all_lme_run,vert=True)
plt.xticks([])
#plt.ylim(-1,1)
plt.title('Boxplot')
plt.ylabel('lme  coefft',fontsize=20)
plt.xlim(.8,1.2)
plt.ylim(-.006,.006)
plt.tight_layout()


In [ ]:
print(len(np.where(pval_nodes_fdr_allother_lme_run)[0]), 'nodes')

plt.figure(1,figsize=(10,5))
plt.suptitle('ALL OTHER MOVIES, trait-fMRI (slope beta) reg coefft\nacross all 268 nodes')
plt.subplot(121)
plt.hist(coef_nodes_allother_lme_run,bins=20,orientation='horizontal',color='grey')
_, pval = stats.ttest_1samp(coef_nodes_allother_lme_run,0)
avg  = np.nanmean(coef_nodes_allother_lme_run)
xmin,xmax = plt.xlim()
plt.hlines(avg,xmin,xmax,color = 'orange',label = f'Mean= {avg:.4f},\n(p={pval:.1e})')
#plt.ylim(-1,1)
plt.ylabel('lme  coefft',fontsize=20)
plt.xlabel('Nr. of nodes')
plt.title('Histogram')
plt.legend()
plt.ylim(-.006,.006)

plt.subplot(122)
plt.boxplot(coef_nodes_allother_lme_run,vert=True)
plt.xticks([])
#plt.ylim(-1,1)
plt.title('Boxplot')
plt.ylabel('lme  coefft',fontsize=20)
plt.xlim(.8,1.2)
plt.tight_layout()
plt.ylim(-.006,.006)

In [ ]:
print([i for i in behav_data.columns])

# Plot brain maps

In [ ]:
pval_nodes_all_lme_run.shape

In [ ]:
print('all nodes:',min(coef_nodes_all_lme_run),max(coef_nodes_all_lme_run))
print('sig unc nodes:',min(coef_nodes_all_lme_run[pval_nodes_all_lme_run<.05]),max(coef_nodes_all_lme_run[pval_nodes_all_lme_run<.05]))

In [ ]:
# surprising
coords = [-50,-40,-25,-10,0,10,25,40,55] # initial exploration
#coords = [-40,-30,-20] # for the final version
cmap = 'RdBu_r'
   
vmin,vmax = -.006,.006

#txt = r"$\overline{\beta}(''Social''-''Non-social'')$"
txt = 'reg coefft (ASR_Intn_T vs.\nslope beta)'

nodes = np.zeros((268,))
nodes[pval_nodes_all_lme_run<.05] = coef_nodes_all_lme_run[pval_nodes_all_lme_run<.05]
title_txt = f'ALL MOVIES (nodes unc.)'

contours_mat = np.zeros((268,))
contours_mat[pval_nodes_fdr_all_lme_run] = 1
#contours_mat[nodes_coaxbill_rand_all] = 2
contours_mat[nodes_coaxbill_rand_all] = 2

levels = [1,2]
labels = ['q<.05','GLM nodes']
colors = [[0,0,0,1],[.5,0,.5,1]]
params = {'levels':levels, 'labels': labels, 'colors': colors, 'coords':coords}
text_x = .1
plot_result_brainmaps(nodes,vmin,vmax,txt,title_txt,contours_mat,pval_nodes_fdr_all_lme_run,nodes_coaxbill_rand_all,params,text_x)


In [ ]:
[n for n in range(268) if 'amygdala' in str.lower(eval(shen268_lbl[str(n+1)][0])['name']) ]

In [ ]:
nodes = np.where(pval_nodes_all_lme_run<.05)[0]
print(len(nodes))
[n for n in nodes if 'amygdala' in str.lower(eval(shen268_lbl[str(n+1)][0])['name']) ]

In [ ]:

#nodes = np.where(nodes_coaxbill_rand_all & (coef_nodes_all_lme_run>0))[0]
nodes = np.where(pval_nodes_fdr_all_lme_run & (coef_nodes_all_lme_run>0))[0]
#nodes = nodes[nodes>134]
print(len(nodes))

# writing our nodes common for RANDOM MECH and ALL MOVIES
pd.DataFrame({'Node': [n+1 for n in nodes], 'reg coefft':coef_nodes_all_lme_run[nodes],'pval':pval_nodes_all_lme_run[nodes], 'ROI name':[eval(shen268_lbl[str(n+1)][0])['name'] for n in nodes]}).sort_values(by=['reg coefft'],ascending = False)

In [ ]:

#nodes = np.where(nodes_coaxbill_rand_all & (coef_nodes_all_lme_run>0))[0]
nodes = np.where((pval_nodes_all_lme_run<.05) & ~pval_nodes_fdr_all_lme_run & (coef_nodes_all_lme_run>0))[0]
#nodes = nodes[nodes>134]
print(len(nodes))

# writing our nodes common for RANDOM MECH and ALL MOVIES
pd.DataFrame({'Node': [n+1 for n in nodes], 'reg coefft':coef_nodes_all_lme_run[nodes],'pval':pval_nodes_all_lme_run[nodes], 'ROI name':[eval(shen268_lbl[str(n+1)][0])['name'] for n in nodes]}).sort_values(by=['reg coefft'],ascending = False)

In [ ]:
coords = [-50,-40,-25,-10,0,10,25,40] # initial exploration
#coords = [-40,-30,-20] # for the final version
cmap = 'RdBu_r'
   
vmin,vmax = -.006,.006

#txt = r"$\overline{\beta}(''Social''-''Non-social'')$"
txt = 'reg coefft (ASR_Intn_T vs.\nslope beta)'

nodes = np.zeros((268,))
nodes[pval_nodes_all_lme_run<.05] = coef_nodes_all_lme_run[pval_nodes_all_lme_run<.05]
title_txt = f'ALL MOVIES (nodes unc.)'

fig,ax = plt.subplots(nrows=1,ncols=4,figsize = (20,5),subplot_kw={'projection': '3d'})
colors_glm_nodes = [[.1,.1,.1,1]]
colors_fdr = ['k']#[[0,.5,0,1]]

img = roi_to_brain(pd.Series(nodes), mask_x)
ax_plot = plot_img(img.to_nifti(), display_mode = 'z',vmin = vmin, vmax = vmax, cut_coords =coords,cmap = cmap, bg_img = bg_img,colorbar= False,threshold = .0001)
#ax_plot.add_contours(color_rois((nodes_coaxbill_rand_all)),linewidths=1, colors=colors_glm_nodes,linestyles='solid')
ax_plot.add_contours(color_rois((pval_nodes_fdr_all_lme_run)),linewidths=1, colors=colors_fdr,linestyles='solid')
#ax_plot.title(title_txt)
#ax_plot._colorbar_ax.text(1,.8*vmax,txt,fontsize=20,fontdict = {'verticalalignment':'top','rotation':0})#get_legend()
#plt.savefig(os.path.join(fig_save_loc,f'RAND/{resp_lbl[resp]}/axial.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
#plt.clf()

#LH
texture = vol_to_surf(color_rois(nodes), fsaverage.pial_left,interpolation='nearest',radius =1, n_samples=1)
surf_plot1=plot_surf_roi(fsaverage.infl_left, texture, hemi='left',cmap = cmap, colorbar=False, symmetric_cmap=True, vmin = vmin, vmax = vmax,
                            bg_map=fsaverage.sulc_left,axes=ax[0], title=title_txt)#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})

#surf_plot1.axes[0].set_title(title_txt,fontsize=24,color='k',fontweight='bold')
#texture_contour1 = vol_to_surf(color_rois(nodes_coaxbill_rand_all), fsaverage.pial_left,interpolation='nearest',radius =1, n_samples=1)
#plot_surf_contours(fsaverage.infl_left, texture_contour1, axes = ax[0], figure=surf_plot1, legend=True,levels = [1], colors=colors_glm_nodes)
texture_contour2 = vol_to_surf(color_rois(pval_nodes_fdr_all_lme_run), fsaverage.pial_left,interpolation='nearest',radius =1, n_samples=1)
plot_surf_contours(fsaverage.infl_left, texture_contour2, axes = ax[0], figure=surf_plot1, legend=True,levels = [1], colors=colors_fdr)
#plt.savefig(os.path.join(fig_save_loc,f'RAND/{resp_lbl[resp]}/left_lat.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')   
#plt.clf()
   

surf_plot2=plot_surf_roi(fsaverage.infl_left, texture, hemi='left',cmap = cmap, colorbar=False, symmetric_cmap=True, vmin = vmin, vmax = vmax,
                            bg_map=fsaverage.sulc_left, view = 'medial',axes=ax[1])#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
#plot_surf_contours(fsaverage.infl_left, texture_contour1, axes = ax[1], figure=surf_plot2, legend=True,levels = [1], colors=colors_glm_nodes)
plot_surf_contours(fsaverage.infl_left, texture_contour2, axes = ax[1], figure=surf_plot2, legend=True,levels = [1], colors=colors_fdr)
#plt.savefig(os.path.join(fig_save_loc,f'RAND/{resp_lbl[resp]}/left_med.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
#plt.clf()

#RH
texture = vol_to_surf(color_rois(nodes), fsaverage.pial_right,interpolation='nearest',radius =1, n_samples=1)
surf_plot3=plot_surf_roi(fsaverage.infl_right, texture, hemi='right',cmap = cmap, colorbar=True,symmetric_cmap=True, vmin = vmin, vmax = vmax,
                            bg_map=fsaverage.sulc_right,axes=ax[3])#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
surf_plot3.axes[4].text(.1,.5*vmax,s=txt,fontsize=20, fontdict = {'verticalalignment':'top','horizontalalignment':'left','rotation':0})
#texture_contour1 = vol_to_surf(color_rois(nodes_coaxbill_rand_all), fsaverage.pial_right,interpolation='nearest',radius =1, n_samples=1)
#plot_surf_contours(fsaverage.infl_right, texture_contour1, axes = ax[3],figure=surf_plot3, legend=True,levels = [1], colors=colors_glm_nodes) # ,labels=['GLM node']
texture_contour2 = vol_to_surf(color_rois(pval_nodes_fdr_all_lme_run), fsaverage.pial_right,interpolation='nearest',radius =1, n_samples=1)
plot_surf_contours(fsaverage.infl_right, texture_contour2, axes = ax[3],figure=surf_plot3, legend=True,levels = [1], labels=['q<..05'],colors=colors_fdr)
#plt.savefig(os.path.join(fig_save_loc,f'RAND/{resp_lbl[resp]}/right_lat.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
#plt.clf()


surf_plot4=plot_surf_roi(fsaverage.infl_right, texture, hemi='right',cmap = cmap, colorbar=False,symmetric_cmap=True, vmin = vmin, vmax = vmax,
                            bg_map=fsaverage.sulc_right, view ='medial',axes=ax[2])#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
#plot_surf_contours(fsaverage.infl_right, texture_contour1, axes = ax[2], figure=surf_plot4, legend=True,levels = [1], colors=colors_glm_nodes)
plot_surf_contours(fsaverage.infl_right, texture_contour2, axes = ax[2], figure=surf_plot4, legend=True,levels = [1], colors=colors_fdr)
#plt.savefig(os.path.join(fig_save_loc,f'RAND/{resp_lbl[resp]}/right_med.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
#plt.clf()


In [ ]:
print('all nodes:',min(coef_nodes_allother_lme_run),max(coef_nodes_allother_lme_run))
print('sig unc nodes:',min(coef_nodes_allother_lme_run[pval_nodes_allother_lme_run<.05]),max(coef_nodes_allother_lme_run[pval_nodes_allother_lme_run<.05]))

In [ ]:
len(np.where(pval_nodes_fdr_allother_lme_run)[0]),len(np.where(pval_nodes_fdr_movie)[0]), len(np.where(pval_nodes_fdr_allother_lme_run & pval_nodes_fdr_movie)[0])

In [ ]:
coords = [-50,-40,-25,-10,0,10,25,40] # initial exploration
#coords = [-40,-30,-20] # for the final version
cmap = 'RdBu_r'
   
vmin,vmax = -.006,.006

#txt = r"$\overline{\beta}(''Social''-''Non-social'')$"
txt = 'reg coefft (ASR_Intn_T vs.\nslope beta)'

nodes = np.zeros((268,))
nodes[pval_nodes_allother_lme_run<.05] = coef_nodes_allother_lme_run[pval_nodes_allother_lme_run<.05]
title_txt = f'ALL EXCEPT SURPRISING-B (nodes unc.)'

fig,ax = plt.subplots(nrows=1,ncols=4,figsize = (20,5),subplot_kw={'projection': '3d'})
colors_glm_nodes = [[.1,.1,.1,1]]
colors_fdr = ['k']#[[0,.5,0,1]]

img = roi_to_brain(pd.Series(nodes), mask_x)
ax_plot = plot_img(img.to_nifti(), display_mode = 'z',vmin = vmin, vmax = vmax, cut_coords =coords,cmap = cmap, bg_img = bg_img,colorbar= False,threshold = .0001)
#ax_plot.add_contours(color_rois((nodes_coaxbill_rand_all)),linewidths=1, colors=colors_glm_nodes,linestyles='solid')
ax_plot.add_contours(color_rois((pval_nodes_fdr_allother_lme_run)),linewidths=1, colors=colors_fdr,linestyles='solid')
#ax_plot.title(title_txt)
#ax_plot._colorbar_ax.text(1,.8*vmax,txt,fontsize=20,fontdict = {'verticalalignment':'top','rotation':0})#get_legend()
#plt.savefig(os.path.join(fig_save_loc,f'RAND/{resp_lbl[resp]}/axial.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
#plt.clf()

#LH
texture = vol_to_surf(color_rois(nodes), fsaverage.pial_left,interpolation='nearest',radius =1, n_samples=1)
surf_plot1=plot_surf_roi(fsaverage.infl_left, texture, hemi='left',cmap = cmap, colorbar=False, symmetric_cmap=True, vmin = vmin, vmax = vmax,
                            bg_map=fsaverage.sulc_left,axes=ax[0], title=title_txt)#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})

#surf_plot1.axes[0].set_title(title_txt,fontsize=24,color='k',fontweight='bold')
#texture_contour1 = vol_to_surf(color_rois(nodes_coaxbill_rand_all), fsaverage.pial_left,interpolation='nearest',radius =1, n_samples=1)
#plot_surf_contours(fsaverage.infl_left, texture_contour1, axes = ax[0], figure=surf_plot1, legend=True,levels = [1], colors=colors_glm_nodes)
texture_contour2 = vol_to_surf(color_rois(pval_nodes_fdr_allother_lme_run), fsaverage.pial_left,interpolation='nearest',radius =1, n_samples=1)
plot_surf_contours(fsaverage.infl_left, texture_contour2, axes = ax[0], figure=surf_plot1, legend=True,levels = [1], colors=colors_fdr)
#plt.savefig(os.path.join(fig_save_loc,f'RAND/{resp_lbl[resp]}/left_lat.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')   
#plt.clf()
   

surf_plot2=plot_surf_roi(fsaverage.infl_left, texture, hemi='left',cmap = cmap, colorbar=False, symmetric_cmap=True, vmin = vmin, vmax = vmax,
                            bg_map=fsaverage.sulc_left, view = 'medial',axes=ax[1])#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
#plot_surf_contours(fsaverage.infl_left, texture_contour1, axes = ax[1], figure=surf_plot2, legend=True,levels = [1], colors=colors_glm_nodes)
plot_surf_contours(fsaverage.infl_left, texture_contour2, axes = ax[1], figure=surf_plot2, legend=True,levels = [1], colors=colors_fdr)
#plt.savefig(os.path.join(fig_save_loc,f'RAND/{resp_lbl[resp]}/left_med.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
#plt.clf()

#RH
texture = vol_to_surf(color_rois(nodes), fsaverage.pial_right,interpolation='nearest',radius =1, n_samples=1)
surf_plot3=plot_surf_roi(fsaverage.infl_right, texture, hemi='right',cmap = cmap, colorbar=True,symmetric_cmap=True, vmin = vmin, vmax = vmax,
                            bg_map=fsaverage.sulc_right,axes=ax[3])#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
surf_plot3.axes[4].text(.1,.5*vmax,s=txt,fontsize=20, fontdict = {'verticalalignment':'top','horizontalalignment':'left','rotation':0})
#texture_contour1 = vol_to_surf(color_rois(nodes_coaxbill_rand_all), fsaverage.pial_right,interpolation='nearest',radius =1, n_samples=1)
#plot_surf_contours(fsaverage.infl_right, texture_contour1, axes = ax[3],figure=surf_plot3, legend=True,levels = [1], colors=colors_glm_nodes) # ,labels=['GLM node']
texture_contour2 = vol_to_surf(color_rois(pval_nodes_fdr_allother_lme_run), fsaverage.pial_right,interpolation='nearest',radius =1, n_samples=1)
plot_surf_contours(fsaverage.infl_right, texture_contour2, axes = ax[3],figure=surf_plot3, legend=True,levels = [1], labels=['q<..05'],colors=colors_fdr)
#plt.savefig(os.path.join(fig_save_loc,f'RAND/{resp_lbl[resp]}/right_lat.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
#plt.clf()

surf_plot4=plot_surf_roi(fsaverage.infl_right, texture, hemi='right',cmap = cmap, colorbar=False,symmetric_cmap=True, vmin = vmin, vmax = vmax,
                            bg_map=fsaverage.sulc_right, view ='medial',axes=ax[2])#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
#plot_surf_contours(fsaverage.infl_right, texture_contour1, axes = ax[2], figure=surf_plot4, legend=True,levels = [1], colors=colors_glm_nodes)
plot_surf_contours(fsaverage.infl_right, texture_contour2, axes = ax[2], figure=surf_plot4, legend=True,levels = [1], colors=colors_fdr)
#plt.savefig(os.path.join(fig_save_loc,f'RAND/{resp_lbl[resp]}/right_med.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
#plt.clf()


In [ ]:
[col for col in behav_data if col.startswith('ASR_Intn')]

In [ ]:
behav_data.columns

In [ ]:
def intersection(lst1, lst2):
    
    if type(lst1) != 'list':
        lst1 = list(lst1)
    
    if type(lst2) != 'list':
        lst2 = list(lst2)
    
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

nodes = intersection((np.where(pval_nodes_rand<.05)[0]),(np.where(pval_nodes_fdr_all_lme_run)[0]))

# writing our nodes common for RANDOM MECH and ALL MOVIES
pd.DataFrame({'Node': [n+1 for n in nodes], 'rcoef(Sp)':corr_nodes_rand[nodes],'pval(Sp)':pval_nodes_rand[nodes],  'rcoef(lme)':coef_nodes_all_lme_run[nodes],'pval(lme)':pval_nodes_all_lme_run[nodes], 
'ROI name':[eval(shen268_lbl[str(n+1)][0])['name'] for n in nodes]}).sort_values(by=['rcoef(Sp)'],
ascending = False)


In [ ]:

nodes2 = intersection((np.where(pval_nodes_rand<.05)[0]),(np.where(pval_nodes_all_lme_run)[0]))

# writing our nodes common for RANDOM MECH and ALL MOVIES
pd.DataFrame({'Node': [n+1 for n in nodes2], 'RANDOM MECH rcoef(Sp)':corr_nodes_rand[nodes2],'RANDOM MECH pval(Sp)':pval_nodes_rand[nodes2],  'ALL rcoef(lme)':coef_nodes_all_lme_run[nodes2],'ALL pval(lme)':pval_nodes_all_lme_run[nodes2], 
'ROI name':[eval(shen268_lbl[str(n+1)][0])['name'] for n in nodes2]}).sort_values(by=['RANDOM MECH rcoef(Sp)'],
ascending = False)

In [ ]:
x,y = stats.zscore(corr_nodes_rand),stats.zscore(coef_nodes_all_lme_run)

fig1 = plt.figure(1,figsize=(4,4))
ax = fig1.add_subplot(111)
plt.scatter(x,y,marker='.',color='grey')
plt.scatter(x[nodes],y[nodes],marker='o',color='k')
ax.set_aspect('equal')
plt.xlim(-5,5)
plt.ylim(-5,5)
plt.xlabel('Z(corr coef.\nRANDOM MECH)')
plt.ylabel('Z(LME coef.\nALL MOVIES)')
rs,ps =  stats.spearmanr(corr_nodes_rand,coef_nodes_all_lme_run)
plt.annotate(f'r_s = {rs:.2f}\n(p={ps:.1g})', xy=(.05,.8), xycoords='axes fraction')
plt.xticks([-5,-2.5,0,2.5,5])
plt.yticks([-5,-2.5,0,2.5,5])

# additional exploratory analysis

In [ ]:
all_coefs_run.shape

In [ ]:
# all

corrtype='Spearman' # 'Pearson'

x = "slopeReg_node"
#y = "Loneliness_Unadj"
y = "ASR_Intn_T"

corr_nodes_all_sep,    pval_nodes_all_sep   =  [np.empty((268,10)) for _ in range(2)] # nodes * responses (non-social, social)
corr_nodes_all_sep[:], pval_nodes_all_sep[:] = [np.nan] *2

pval_nodes_fdr_all_sep = np.empty_like(pval_nodes_all_sep)
pval_nodes_fdr_all_sep[:] = np.nan

#rtemp,   ptemp    =  [np.empty((268,10)) for _ in range(2)] # nodes * responses (non-social, social)
#rtemp[:],ptemp[:] = [np.nan] *2

for n in range(268): 
    if n% 50 ==0:
        print('node',n)

    for m in range(10):

        fMRI_data = pd.DataFrame({'Subject':sub_id_all,'slopeReg_node':all_coefs_run[:,n,m], 'responses':responses[:,m]})
        fMRI_data.set_index("Subject", inplace=True)
        data = fMRI_data.join(behav_data, how='inner') # join betas and trait info
        
        inds = ~np.isnan(data[x]) & ~np.isnan(data[y]) #& ~np.isnan(data['responses'])# find rows where neither x or y is NaN
        
        # Calculate and print correlations
        if corrtype == 'Pearson':
            corr_nodes_all_sep[n,m], pval_nodes_all_sep[n,m] = stats.pearsonr(data[x][inds], data[y][inds])
        elif corrtype =='Spearman':
            corr_nodes_all_sep[n,m], pval_nodes_all_sep[n,m] = stats.spearmanr(data[x][inds], data[y][inds])
            #rtemp[n,m],ptemp[n,m] = stats.pearsonr(data[x][inds], data[y][inds])

for m in range(10):
    pval_nodes_fdr_all_sep[:,m] = lsu(pval_nodes_all_sep[:,m],q=.05)
    #ptemp_fdr = lsu(ptemp,q=.05)


In [ ]:
pval_nodes_rand

In [ ]:
pval_nodes_all_sep[:,8]

In [ ]:
 lsu(pval_nodes_all_sep[:,8],q=.05)

In [ ]:
for m in range(10):
    print('movie',m+1,np.where(pval_nodes_fdr_all_sep[:,m]))

In [ ]:
len(np.where(pval_nodes_fdr_all_lme_run)[0])

In [ ]:
np.where(pval_nodes_fdr_all_sep)

In [ ]:
# nodes which show a diff between corr"Social" and corr"Non-social" for RANODM MECH and across nodes in the main GLM analysis
print('movie 8')
print('overlap with the main GLM nodes: unc.', len(np.where(nodes_coaxbill_rand_all & (pval_nodes_all[:,8]<.05))[0]) )
print('overlap with the main GLM nodes: fdr.', len(np.where(nodes_coaxbill_rand_all & (pval_nodes_fdr_all[:,8]==1))[0]) )

In [ ]:
plt.hist(corr_nodes_all[(nodes_coaxbill_rand_all) & (pval_nodes_all[:,8]<.05),8])

# extra code

In [ ]:
coef_slope_rand = np.load(os.path.join(data_file_loc,'coefs_run_norm','slope_reg','coef_slopereg_runnorm_Random mechanical.npy'))
coef_slope_fish = np.load(os.path.join(data_file_loc,'coefs_run_norm','slope_reg','coef_slopereg_runnorm_Fishing.npy'))
coef_slope_coax = np.load(os.path.join(data_file_loc,'coefs_trial_norm','slope_reg','coef_slopereg_trialnorm_COAXING-B.npy'))
coef_slope_bill = np.load(os.path.join(data_file_loc,'coefs_trial_norm','slope_reg','coef_slopereg_trialnorm_BILLIARD-A.npy'))
coef_slope_all  = np.load(os.path.join(data_file_loc,'coef_slopereg_all.npy'))

print(coef_slope_rand.shape)
coef_slope_all.shape

In [ ]:
print(np.where(coef_nodes_all_lme_run == max(coef_nodes_all_lme_run)),max(coef_nodes_all_lme_run))
print(np.where(coef_nodes_all_lme_run == min(coef_nodes_all_lme_run)),min(coef_nodes_all_lme_run))

data = pd.DataFrame({})#columns = ['Subject','coef','response','movie'])
n = 49
for m in range(10):
    #fMRI_data = pd.DataFrame({'coefs': all_coefs[subs_10resp,n,m],'response':responses[subs_10resp,m], 'subID':sub_id_all[subs_10resp], 'movie':np.repeat(m,len(subs_10resp))})
    fMRI_data = pd.DataFrame({'Subject':sub_id_all,'slopeReg_node': all_coefs_run[:,n,m], 'response': responses[:,m],'nresp':nresp, 'movie': np.repeat(m,1048)})
    #fMRI_data = fMRI_data.loc[fMRI_data['nresp']==10,:]
    fMRI_data.set_index("Subject", inplace=True)
    data_temp = fMRI_data.join(behav_data['ASR_Intn_T'], how='inner') # join betas and trait info
    data = data.append(data_temp)#,ignore_index=True)
data.reset_index(inplace=True)

inds = ~np.isnan(data['slopeReg_node']) & ~np.isnan(data['ASR_Intn_T']) # find rows where neither x or y is NaN
data = data.loc[inds,:]
data['asr_z'] = stats.zscore(data['ASR_Intn_T'].values)
explore_correlation('ASR_Intn_T','slopeReg_node',data)
explore_correlation('asr_z','slopeReg_node',data)

model1 = Lmer('slopeReg_node ~  ASR_Intn_T + (1|movie)', data=data) # ff: mean response, rf:subjID
model1.fit(summary = False, verbose = False, no_warnings = True)

model2 = Lmer('slopeReg_node ~  asr_z + (1|movie)', data=data) # ff: mean response, rf:subjID
model2.fit(summary = False, verbose = False, no_warnings = True)
#model.fit(factors={"response":["Non-social","Social"]},summary = False, verbose = False)

#if (len(model1.warnings) == 0):
#    coef_nodes_all_lr_run[n] = model1.coefs['Estimate'][1]
#    pval_nodes_all_lr_run[n] = model1.coefs['P-val'][1]

if (len(model1.warnings) == 0):
    coef_1 = model1.coefs['Estimate'][1]
    pval_1 = model1.coefs['P-val'][1]

if (len(model2.warnings) == 0):
    coef_2 = model2.coefs['Estimate'][1]
    pval_2 = model2.coefs['P-val'][1]

In [ ]:
def plot_brain_maps(nodes,vmin,vmax,txt,title_txt,contours_mat):
    coords = [-50,-40,-25,-10,0,10,25,40] # initial exploration
    #coords = [-40,-30,-20] # for the final version
    cmap = 'RdBu_r'

    colors = [[0,0,0,1],[0,1,0,1],[1,0,0,1]]


    fig,ax = plt.subplots(nrows=1,ncols=4,figsize = (20,5),subplot_kw={'projection': '3d'})
colors_glm_nodes = [[.1,.1,.1,1]]
colors_fdr = ['k']#[[0,.5,0,1]]
colors_intersection = ['r']

colors = [[0,0,0,1],[0,1,0,1],[1,0,0,1]]

img = roi_to_brain(pd.Series(nodes), mask_x)
ax_plot = plot_img(img.to_nifti(), display_mode = 'z',vmin = vmin, vmax = vmax, cut_coords =coords,cmap = cmap, bg_img = bg_img,colorbar= False,threshold = .0001)
#ax_plot.add_contours(color_rois((nodes_coaxbill_rand_all)),linewidths=1, colors=colors_glm_nodes,linestyles='solid')
ax_plot.add_contours(color_rois((pval_nodes_fdr_surp)),linewidths=1, colors=colors_fdr,linestyles='solid')
#ax_plot.title(title_txt)
#ax_plot._colorbar_ax.text(1,.8*vmax,txt,fontsize=20,fontdict = {'verticalalignment':'top','rotation':0})#get_legend()
#plt.savefig(os.path.join(fig_save_loc,f'RAND/{resp_lbl[resp]}/axial.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
#plt.clf()

#LH
texture = vol_to_surf(color_rois(nodes), fsaverage.pial_left,interpolation='nearest',radius =1, n_samples=1)
surf_plot1=plot_surf_roi(fsaverage.infl_left, texture, hemi='left',cmap = cmap, colorbar=False, symmetric_cmap=True, vmin = vmin, vmax = vmax,
                            bg_map=fsaverage.sulc_left,axes=ax[0], title=title_txt)#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
#surf_plot1.axes[0].set_title(title_txt,fontsize=24,color='k',fontweight='bold')
#texture_contour1 = vol_to_surf(color_rois(nodes_coaxbill_rand_all), fsaverage.pial_left,interpolation='nearest',radius =1, n_samples=1)
#plot_surf_contours(fsaverage.infl_left, texture_contour1, axes = ax[0], figure=surf_plot1, legend=True,levels = [1], colors=colors_glm_nodes)
texture_contour2 = vol_to_surf(color_rois(contours_mat), fsaverage.pial_left,interpolation='nearest',radius =1, n_samples=1)
plot_surf_contours(fsaverage.infl_left, texture_contour2, axes = ax[0], figure=surf_plot1, legend=True,levels = [1,2,3], colors=colors)
#texture_contour2 = vol_to_surf(color_rois(pval_nodes_fdr_surp), fsaverage.pial_left,interpolation='nearest',radius =1, n_samples=1)
#plot_surf_contours(fsaverage.infl_left, texture_contour2, axes = ax[0], figure=surf_plot1, legend=True,levels = [1], colors=colors_fdr)
#plt.savefig(os.path.join(fig_save_loc,f'RAND/{resp_lbl[resp]}/left_lat.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')   
#plt.clf()
   

surf_plot2=plot_surf_roi(fsaverage.infl_left, texture, hemi='left',cmap = cmap, colorbar=False, symmetric_cmap=True, vmin = vmin, vmax = vmax,
                            bg_map=fsaverage.sulc_left, view = 'medial',axes=ax[1])#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
#plot_surf_contours(fsaverage.infl_left, texture_contour1, axes = ax[1], figure=surf_plot2, legend=True,levels = [1], colors=colors_glm_nodes)
#plot_surf_contours(fsaverage.infl_left, texture_contour2, axes = ax[1], figure=surf_plot2, legend=True,levels = [1], colors=colors_fdr)
plot_surf_contours(fsaverage.infl_left, texture_contour2, axes = ax[1], figure=surf_plot2, legend=True,levels = [1,2,3], colors=colors)
#plt.savefig(os.path.join(fig_save_loc,f'RAND/{resp_lbl[resp]}/left_med.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
#plt.clf()

#RH
texture = vol_to_surf(color_rois(nodes), fsaverage.pial_right,interpolation='nearest',radius =1, n_samples=1)
surf_plot3=plot_surf_roi(fsaverage.infl_right, texture, hemi='right',cmap = cmap, colorbar=True,symmetric_cmap=True, vmin = vmin, vmax = vmax,
                            bg_map=fsaverage.sulc_right,axes=ax[3])#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
surf_plot3.axes[4].text(2,.5*vmax,s=txt,fontsize=20, fontdict = {'verticalalignment':'top','horizontalalignment':'left','rotation':0})
#texture_contour1 = vol_to_surf(color_rois(nodes_coaxbill_rand_all), fsaverage.pial_right,interpolation='nearest',radius =1, n_samples=1)
#plot_surf_contours(fsaverage.infl_right, texture_contour1, axes = ax[3],figure=surf_plot3, legend=True,levels = [1], colors=colors_glm_nodes) # ,labels=['GLM node']
#texture_contour2 = vol_to_surf(color_rois(pval_nodes_fdr_surp), fsaverage.pial_right,interpolation='nearest',radius =1, n_samples=1)
texture_contour2 = vol_to_surf(color_rois(contours_mat), fsaverage.pial_right,interpolation='nearest',radius =1, n_samples=1)
plot_surf_contours(fsaverage.infl_right, texture_contour2, axes = ax[3], figure=surf_plot4, legend=True,levels = [1,2,3], colors=colors)
#plot_surf_contours(fsaverage.infl_right, texture_contour2, axes = ax[3],figure=surf_plot3, legend=True,levels = [1], labels=['q<..05'],colors=colors_fdr)
#plt.savefig(os.path.join(fig_save_loc,f'RAND/{resp_lbl[resp]}/right_lat.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
#plt.clf()


surf_plot4=plot_surf_roi(fsaverage.infl_right, texture, hemi='right',cmap = cmap, colorbar=False,symmetric_cmap=True, vmin = vmin, vmax = vmax,
                            bg_map=fsaverage.sulc_right, view ='medial',axes=ax[2])#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
#plot_surf_contours(fsaverage.infl_right, texture_contour1, axes = ax[2], figure=surf_plot4, legend=True,levels = [1], colors=colors_glm_nodes)
#plot_surf_contours(fsaverage.infl_right, texture_contour2, axes = ax[2], figure=surf_plot4, legend=True,levels = [1], colors=colors_fdr)
plot_surf_contours(fsaverage.infl_right, texture_contour2, axes = ax[2], figure=surf_plot4, legend=True,levels = [1,2,3], colors=colors)
#plt.savefig(os.path.join(fig_save_loc,f'RAND/{resp_lbl[resp]}/right_med.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
#plt.clf()
